In [3]:
import torch
from torchvision import transforms
from PIL import Image
from model.edgevit import *
from model.mobilenetv2 import *
import pandas as pd
import os
import numpy as np
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 定义加载模型的函数  
def load_model(model_path):  
    # 加载模型  
    model = MobileNetV2().to(device)  # 替换为你的模型类
    checkpoint = torch.load(model_path, map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])  # 加载模型权重  
    model.eval()  # 设置模型为评估模式  
    return model 

# 定义推理函数  
def inference(model, image_path):  
    # 图像预处理  
    mytransform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456,0.406), std=(0.229, 0.224, 0.225))
    ])
    image = Image.open(image_path)
    input_tensor = mytransform(image).to(device)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model  
  
    # 推理  
    with torch.no_grad():  
        output = model(input_batch)  
    return output


def get_true_tiller(img_filename):
        csv_path = "datasets/test/test_2.csv"
        df = pd.read_csv(csv_path)
        column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name



        if hasattr(re.search(r'_(.*?)_', column_name), 'group'):  
            # 如果属性存在，则安全地访问它
            # print(f"{img} OKOKOKOKOKOKOK")
            day_number = re.search(r'_(.*?)_', column_name).group(1) 
            # 接下来使用value进行其他操作...  
        else:  
            # 如果属性不存在，则处理异常或记录错误  
            print(f"\"{img}\",")  
            # 可以选择抛出异常、记录日志或进行其他错误处理
        # day_number = re.search(r'_(.*?)_', column_name).group(1) # from column name get its day number(to find if useful)


#######################################################
        row_indice = np.where(df[column_name] == img_filename)[0][0] # from column name get its row indice

        # category = df.species[row_indice]
        tiller_column_name = "day_" + day_number
        tiller_num = df[tiller_column_name][row_indice]
        return tiller_num

def show_tiller_difference(img_filename):
    model = load_model("./saved_weights/mobilenet/model_epoch_30.pth")
    image = os.path.join("datasets/test/photos", img_filename)
    a = inference(model, image).item()
    b = get_true_tiller(img_filename)
    print("True tiller: {}, Inference tiller: {}, diff: {}".format(b,a, a-b))

In [4]:

img_filenames = ["IMG_3343.JPG", "IMG_3502.JPG", "IMG_3503.JPG", "IMG_3505.JPG", "IMG_3507.JPG", "IMG_3508.JPG","DSC_0687.JPG"]
for img_filename in img_filenames:
    show_tiller_difference(img_filename)



/tmp/ipykernel_247560/196580401.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name


True tiller: 43, Inference tiller: 159.650634765625, diff: 116.650634765625


/tmp/ipykernel_247560/196580401.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name


True tiller: 79, Inference tiller: 158.41722106933594, diff: 79.41722106933594


/tmp/ipykernel_247560/196580401.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name


...

/tmp/ipykernel_247560/196580401.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name


True tiller: 75, Inference tiller: 158.62905883789062, diff: 83.62905883789062


/tmp/ipykernel_247560/196580401.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name


True tiller: 73, Inference tiller: 159.48849487304688, diff: 86.48849487304688
True tiller: 358, Inference tiller: 160.64077758789062, diff: -197.35922241210938


/tmp/ipykernel_247560/196580401.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  column_name = df.applymap(lambda x: x == img_filename).any().idxmax()   #from elemant content search its column name


In [35]:
l1_reg = 0
model = load_model("./saved_weights/edgevits/model_epoch_40_onehot.pth")
for param in model.parameters():  
        l1_reg += torch.norm(param, 1)  
l1_reg

tensor(287681.4688, device='cuda:0', grad_fn=<AddBackward0>)